## 2 Preparação do Ambiente

In [1]:
# Instalação das bibliotecas (execute apenas se não estiverem instaladas)
# !pip install gensim matplotlib scikit-learn pandas numpy spacy plotly

# Importações básicas
import numpy as np # trabalhar matrizes
import pandas as pd # dataframes
import matplotlib.pyplot as plt # gráficos
from gensim.models import Word2Vec # modelo de vetorização
import gensim.downloader as api
from sklearn.manifold import TSNE # framework de treinamento
import re #regex
import nltk # trabalho de frases e palavras
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

# Download de recursos NLTK (se necessário)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

##  3 Preparando um Dataset para Treinamento
### 3.1 Carregando Dados de Exemplo

In [4]:
# Dados de exemplo - críticas de filmes (simplificadas)
textos = [
    "Este filme é incrível, adorei a atuação do protagonista",
    "A direção de fotografia é espetacular e o roteiro é envolvente",
    "Péssimo filme, desperdicei meu tempo assistindo isso",
    "Os atores são talentosos mas o roteiro é fraco",
    "Cinematografia belíssima, recomendo assistir no cinema",
    "Não gostei da história, personagens mal desenvolvidos",
    "A trilha sonora combina perfeitamente com as cenas",
    "Filme entediante, previsível do início ao fim",
    "Os efeitos especiais são impressionantes, tecnologia de ponta",
    "História emocionante, chorei no final do filme"
]

# Verificando os dados
for i, texto in enumerate(textos[:3]):  # Mostrando apenas os 3 primeiros
    print(f"Texto {i+1}: {texto}")

Texto 1: Este filme é incrível, adorei a atuação do protagonista
Texto 2: A direção de fotografia é espetacular e o roteiro é envolvente
Texto 3: Péssimo filme, desperdicei meu tempo assistindo isso


### 3.2 Pré-processamento do Texto

#### Antes de gerar embeddings, precisamos pré-processar os textos:

In [5]:
from nltk.corpus import stopwords

def preprocessar_texto(texto):
    # Converter para minúsculas
    texto = texto.lower()

    # Remover caracteres especiais e números
    texto = re.sub(r'[^a-záàâãéèêíïóôõöúçñ ]', '', texto)

    # Tokenizar
    tokens = word_tokenize(texto)

    # Remover stopwords (opcional, dependendo da aplicação)
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

# Aplicar pré-processamento a todos os textos
textos_preprocessados = [preprocessar_texto(texto) for texto in textos]

# Verificar resultado
print("Exemplo de texto original:")
print(textos[0])
print("\nDepois do pré-processamento:")
print(textos_preprocessados[0])

Exemplo de texto original:
Este filme é incrível, adorei a atuação do protagonista

Depois do pré-processamento:
['filme', 'incrível', 'adorei', 'atuação', 'protagonista']


## 4. Implementando Word2Vec com Gensim
#### 4.1 Treinando um Modelo do Zero
#### O que é a dimensionalidade?
#### Vamos treinar um modelo Word2Vec simples com nossos dados:


In [6]:
# Definir parâmetros do modelo
vector_size = 100    # Dimensionalidade dos vetores
window = 5           # Tamanho da janela de contexto
min_count = 1        # Frequência mínima das palavras
workers = 4          # Número de threads para treinamento
sg = 1               # Modelo Skip-gram (1) ou CBOW (0)


In [7]:
# Treinar o modelo
model = Word2Vec(
    sentences=textos_preprocessados,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    sg=sg
)

print(f"Modelo treinado com {len(model.wv.key_to_index)} palavras no vocabulário")


Modelo treinado com 44 palavras no vocabulário


### 4.2 Explorando o Modelo Treinado

#### Agora que temos nosso modelo, vamos explorar as palavras e seus embeddings:

In [8]:
# Listar algumas palavras do vocabulário
palavras = list(model.wv.key_to_index.keys())
print("Algumas palavras do vocabulário:")
print(palavras[:10])  # Primeiras 10 palavras

Algumas palavras do vocabulário:
['filme', 'história', 'roteiro', 'desperdicei', 'belíssima', 'cinematografia', 'fraco', 'talentosos', 'atores', 'assistindo']


In [10]:
model.wv.key_to_index

{'filme': 0,
 'história': 1,
 'roteiro': 2,
 'desperdicei': 3,
 'belíssima': 4,
 'cinematografia': 5,
 'fraco': 6,
 'talentosos': 7,
 'atores': 8,
 'assistindo': 9,
 'tempo': 10,
 'péssimo': 11,
 'assistir': 12,
 'envolvente': 13,
 'espetacular': 14,
 'fotografia': 15,
 'direção': 16,
 'protagonista': 17,
 'atuação': 18,
 'adorei': 19,
 'incrível': 20,
 'recomendo': 21,
 'final': 22,
 'chorei': 23,
 'previsível': 24,
 'emocionante': 25,
 'ponta': 26,
 'tecnologia': 27,
 'impressionantes': 28,
 'especiais': 29,
 'efeitos': 30,
 'fim': 31,
 'início': 32,
 'entediante': 33,
 'gostei': 34,
 'cenas': 35,
 'perfeitamente': 36,
 'combina': 37,
 'sonora': 38,
 'trilha': 39,
 'desenvolvidos': 40,
 'mal': 41,
 'personagens': 42,
 'cinema': 43}

In [14]:
if 'cinema' in model.wv:
    vector_filme = model.wv['filme']
    print(vector_filme)

[-5.3898495e-04  2.3754722e-04  5.1084701e-03  9.0095662e-03
 -9.3060182e-03 -7.1179080e-03  6.4590140e-03  8.9732325e-03
 -5.0089178e-03 -3.7637462e-03  7.3839580e-03 -1.5429476e-03
 -4.5339474e-03  6.5549053e-03 -4.8634596e-03 -1.8150365e-03
  2.8793407e-03  9.9878211e-04 -8.2852831e-03 -9.4514769e-03
  7.3103900e-03  5.0675846e-03  6.7592384e-03  7.6005177e-04
  6.3530109e-03 -3.4019963e-03 -9.5264451e-04  5.7722717e-03
 -7.5249239e-03 -3.9323601e-03 -7.5148693e-03 -9.2767383e-04
  9.5453048e-03 -7.3271766e-03 -2.3372965e-03 -1.9302175e-03
  8.0870856e-03 -5.9312731e-03  4.2869444e-05 -4.7512343e-03
 -9.6031642e-03  5.0065401e-03 -8.7594036e-03 -4.3953191e-03
 -3.3294396e-05 -2.9962539e-04 -7.6586660e-03  9.6185729e-03
  4.9843234e-03  9.2339637e-03 -8.1531862e-03  4.4953576e-03
 -4.1356767e-03  8.2382484e-04  8.4956065e-03 -4.4643418e-03
  4.5180386e-03 -6.7829816e-03 -3.5452251e-03  9.4016502e-03
 -1.5767787e-03  3.2035878e-04 -4.1417871e-03 -7.6862508e-03
 -1.5016500e-03  2.46714

In [15]:
# Verificar o vetor de uma palavra específica
if 'filme' in model.wv:
    vetor_filme = model.wv['filme']
    print(f"\nVetor da palavra 'filme' (primeiras 10 dimensões):")
    print(vetor_filme[:10])
    print(f"Dimensionalidade do vetor: {len(vetor_filme)}")
    


Vetor da palavra 'filme' (primeiras 10 dimensões):
[-0.00053898  0.00023755  0.00510847  0.00900957 -0.00930602 -0.00711791
  0.00645901  0.00897323 -0.00500892 -0.00376375]
Dimensionalidade do vetor: 100


### 4.3 Palavras Mais Similares
#### Uma das funcionalidades mais interessantes dos word embeddings é encontrar palavras similares:


In [16]:
# Encontrar palavras mais similares a 'filme'
if 'filme' in model.wv:
    similares = model.wv.most_similar('filme', topn=5)
    print("\nPalavras mais similares a 'filme':")
    for palavra, similaridade in similares:
        print(f"{palavra}: {similaridade:.4f}")



Palavras mais similares a 'filme':
direção: 0.2189
assistindo: 0.2168
impressionantes: 0.1955
cinema: 0.1694
efeitos: 0.1520


### 4.4 Salvando e Carregando o Modelo

#### Para reutilizar nosso modelo futuramente:

In [17]:
model.save("cuiabano.model")

In [18]:
cuiaba = Word2Vec.load("cuiabano.model")

In [19]:
cuiaba

## 5. Usando Modelos Pré-treinados

#### Treinar seu próprio modelo é útil, mas em muitos casos é mais prático usar modelos pré-treinados:

In [20]:
print([nome for nome in list(api.info()['models'].keys())[:10]])

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50']


In [21]:
modelo_pretreinado = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [23]:
# Verificar palavras similares usando modelo pré-treinado
if 'computer' in modelo_pretreinado:
    similares = modelo_pretreinado.most_similar('computer', topn=5)
    print("\nPalavras mais similares a 'computer':")
    for palavra, similaridade in similares:
        print(f"{palavra}: {similaridade:.4f}")


Palavras mais similares a 'computer':
computers: 0.8752
software: 0.8373
technology: 0.7642
pc: 0.7366
hardware: 0.7290


In [24]:
# Usando modelo pré-treinado para analogias (se disponível)
try:
    # Famosa analogia: rei - homem + mulher = rainha
    if all(word in modelo_pretreinado for word in ['king', 'man', 'woman']):
        resultado = modelo_pretreinado.most_similar(
            positive=['king', 'woman'],
            negative=['man'],
            topn=3
        )
        print("\nAnalogia: rei - homem + mulher =")
        for palavra, similaridade in resultado:
            print(f"{palavra}: {similaridade:.4f}")
except:
    print("Não foi possível realizar a operação de analogia com o modelo disponível")

# Podemos tentar outras analogias com nosso modelo treinado
# Por exemplo: bom - positivo + negativo = ruim
# (Dependendo do tamanho do corpus, pode não funcionar bem para modelos pequenos)


Analogia: rei - homem + mulher =
queen: 0.7699
monarch: 0.6843
throne: 0.6756


# 7. Calculando Similaridade entre Palavras

### Vamos explorar como calcular a similaridade entre palavras:

In [ ]:
# Função para calcular similaridade entre pares de palavras
def calcular_similaridade(modelo, pares_palavras):
    resultados = []
    for par in pares_palavras:
        palavra1, palavra2 = par
        if palavra1 in modelo.wv and palavra2 in modelo.wv:
            similaridade = modelo.wv.similarity(palavra1, palavra2)
            resultados.append((par, similaridade))
        else:
            resultados.append((par, "Uma ou ambas as palavras não estão no vocabulário"))
    return resultados

# Pares de palavras para testar
pares = [
    ('filme', 'cinema'),
    ('bom', 'ruim'),
    ('ator', 'atuação'),
    ('filme', 'protagonista')
]

# Calcular similaridades
similaridades = calcular_similaridade(model, pares)

# Exibir resultados
print("\nSimilaridade entre pares de palavras:")
for (palavra1, palavra2), similaridade in similaridades:
    if isinstance(similaridade, float):
        print(f"{palavra1} - {palavra2}: {similaridade:.4f}")
    else:
        print(f"{palavra1} - {palavra2}: {similaridade}")



Similaridade entre pares de palavras:
filme - cinema: 0.1693534404039383
bom - ruim: Uma ou ambas as palavras não estão no vocabulário
ator - teleespectador: Uma ou ambas as palavras não estão no vocabulário
filme - protagonista: -0.11359719187021255


# 9. Aplicação Prática: Classificação de Sentimentos com Embeddings

### Vamos ver como os embeddings podem melhorar um classificador de sentimentos:

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Dados rotulados para exemplo
textos_rotulados = textos  # Usando os mesmos textos de antes
sentimentos = [1, 1, 0, 0, 1, 0, 1, 0, 1, 1]  # 1: positivo, 0: negativo

# Função para gerar vetores de documento usando embeddings
def texto_para_vetor(texto, modelo):
    """Converte um texto em um vetor médio dos embeddings de suas palavras"""
    palavras = preprocessar_texto(texto)
    # Filtrar palavras que estão no vocabulário do modelo
    palavras_no_vocab = [p for p in palavras if p in modelo.wv]
    if not palavras_no_vocab:
        # Se nenhuma palavra estiver no vocabulário, retorna vetor de zeros
        return np.zeros(modelo.vector_size)
    # Calcular a média dos vetores das palavras
    vetores = [modelo.wv[palavra] for palavra in palavras_no_vocab]
    return np.mean(vetores, axis=0)

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    textos_rotulados, sentimentos, test_size=0.3, random_state=42
)

# 1. Abordagem com TF-IDF
vectorizer = TfidfVectorizer(max_features=100)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

clf_tfidf = LogisticRegression(random_state=42)
clf_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)

# 2. Abordagem com Word Embeddings
X_train_emb = np.array([texto_para_vetor(texto, model) for texto in X_train])
X_test_emb = np.array([texto_para_vetor(texto, model) for texto in X_test])

clf_emb = LogisticRegression(random_state=42)
clf_emb.fit(X_train_emb, y_train)
y_pred_emb = clf_emb.predict(X_test_emb)

# Comparar resultados
print("\nResultados com TF-IDF:")
print(classification_report(y_test, y_pred_tfidf))

print("\nResultados com Word Embeddings:")
print(classification_report(y_test, y_pred_emb))



Resultados com TF-IDF:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3


Resultados com Word Embeddings:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3

